In [ ]:
# Staging Metadata Query - stg_metadata.sql

SELECT

    TOPIC as Topic,
    ARRAY_AGG(DISTINCT LEVEL) as Levels,
    ARRAY_AGG(DISTINCT YEAR) as Years,
    COUNT(*) AS number_of_articles

from {{ source('cfa', 'CFA_PROD') }}

GROUP BY TOPIC

In [ ]:
# Staging Summary Query - stg_summary.sql

SELECT
       TOPIC,
       MIN(LENGTH(summary)) AS min_Summary_Length,
       MAX(LENGTH(summary)) AS max_Summary_Length
from {{ source('cfa', 'CFA_PROD') }}
GROUP BY TOPIC


In [ ]:
# Staging Learning Outcomes Query - stg_learning.sql

SELECT
       TOPIC,
       MIN(LENGTH(Learning_Outcomes)) AS min_Learning_Outcome,
       MAX(LENGTH(Learning_Outcomes)) AS max_Learning_Outcome
from {{ source('cfa', 'CFA_PROD') }}
GROUP BY TOPIC


In [ ]:
# Base Model Query - cfa_test.sql

WITH metadata AS (
    SELECT * FROM {{ ref('stg_metadata') }}
),
summary AS (
    SELECT * FROM {{ ref('stg_summary') }}
),
learning AS (
    SELECT * FROM {{ ref('stg_learning') }}
)

SELECT
    m.Topic,
    m.Levels,
    m.Years,
    m.number_of_articles,
    s.min_Summary_Length,
    s.max_Summary_Length,
    l.min_Learning_Outcome,
    l.max_Learning_Outcome
FROM metadata m
JOIN summary s ON m.Topic = s.TOPIC
JOIN learning l ON m.Topic = l.TOPIC

In [ ]:
#Source YAML file for linkage between models - source.yml

version: 2

sources:
    - name: cfa
      description: This is a replica of the Postgres database used by our app
      database: CFA_DATA
      schema: Public
      tables:
          - name: CFA_PROD



In [ ]:
#Test File for the DBT models - schema_test.yml

version: 2

models:
  - name: cfa_test
    description: "The table being considered."

    columns:
      - name: TOPIC
        description: "The primary subject of the CFA articles."
        tests:
          - unique
          - not_null

      - name: NUMBER_OF_ARTICLES
        description: "The count of articles associated with each topic."
        tests:
          - not_null

      - name: YEARS
        description: "An array of years indicating when articles on a topic were published."
        tests:
          - not_null

      - name: MIN_SUMMARY_LENGTH
        description: "The minimum length of article summaries for a given topic."
        tests:
          - not_null

      - name: LEVELS
        description: "An array of CFA levels that the articles pertain to."
        tests:
          - not_null

In [ ]:
# Default YAML file - dbt_project.yml


# Name your project! Project names should contain only lowercase characters
# and underscores. A good package name should reflect your organization's
# name or the intended use of these models
name: 'cfa'
version: '1.0.0'
config-version: 2

# This setting configures which "profile" dbt uses for this project.
profile: 'default'

# These configurations specify where dbt should look for different types of files.
# The `model-paths` config, for example, states that models in this project can be
# found in the "models/" directory. You probably won't need to change these!
model-paths: ["models"]
analysis-paths: ["analyses"]
test-paths: ["tests"]
seed-paths: ["seeds"]
macro-paths: ["macros"]
snapshot-paths: ["snapshots"]

target-path: "target"  # directory which will store compiled SQL files
clean-targets:         # directories to be removed by `dbt clean`
  - "target"
  - "dbt_packages"


# Configuring models
# Full documentation: https://docs.getdbt.com/docs/configuring-models

# In dbt, the default materialization for a model is a view. This means, when you run
# dbt run or dbt build, all of your models will be built as a view in your data platform.
# The configuration below will override this setting for models in the example folder to
# instead be materialized as tables. Any models you add to the root of the models folder will
# continue to be built as views. These settings can be overridden in the individual model files
# using the `{{ config(...) }}` macro.

models:
  cfa:
    +materialized: table
